In [ ]:
# max is cool
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop --quiet

# Download the data folder at https://github.com/goodchemistryco/Tangelo-Examples/tree/main/examples/chemistry/data
import os
if not os.path.isdir("data"):
    !sudo apt install git
    !git clone https://github.com/sandbox-quantum/Tangelo-Examples.git
    !mkdir data
    !cp -a Tangelo-Examples/examples/chemistry/data/. ./data/

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'Tangelo-Examples'...
remote: Enumerating objects: 2251, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 2251 (delta 98), reused 76 (delta 71), pack-reused 2038 (from 1)
Receiving objects: 100% (2251/2251), 10.04 MiB | 22.75 MiB/s, done.
Resolving deltas: 100% (1470/1470), done.


In [ ]:
!pip install --prefer-binary pyscf==2.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 6.2 MB/s eta 0:00:00


In [ ]:
from tangelo import SecondQuantizedMolecule

LiH = [('Li', (0, 0, 0)), ('H', (0, 0, 1.5949))]

mol_LiH = SecondQuantizedMolecule(LiH, q=0, spin=0, basis="6-31g(d,p)")



/usr/local/lib/python3.10/dist-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


Calculate the ground state using classical methods so we have sth to compare the quantum version to.

In [ ]:
from tangelo.algorithms import FCISolver

fci_solver = FCISolver(mol_LiH)
ground_energy= fci_solver.simulate()

Calculate the ground state using VQE. Should we specify the ansatz?? Which one?

In [ ]:
from tangelo.algorithms import VQESolver
from tangelo.toolboxes.ansatz_generator.uccsd import UCCSD
from tangelo.algorithms.variational import BuiltInAnsatze
uccsd_ansatz = UCCSD(molecule=mol_LiH)


vqe_solver = VQESolver({"molecule": mol_LiH, "ansatz": BuiltInAnsatze.UCCSD})
vqe_solver.build()
LiH_qubit_hamiltonian = vqe_solver.qubit_hamiltonian
result = vqe_solver.simulate()
#ground_state =result[1]
# ground_state= wave function but idk how to do that
print("Ground state energy (VQE):", result)

Calculate the excited states.

In [ ]:
# Add initial VQE optimal circuit to the deflation circuits list
deflation_circuits = [vqe_solver.optimal_circuit.copy()]

# Calculate first and second excited states by adding optimal circuits to deflation_circuits
for i in range(2):
    vqe_options = {"molecule": mol_LiH, "ansatz": BuiltInAnsatze.UpCCGSD,
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.4}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver.get_resources()

    deflation_circuits.append(vqe_solver.optimal_circuit.copy())

Time evolution. (Suzuki thing)

In [ ]:
# Dictionary of resources for each algorithm
algorithm_resources = dict()

# Number of Krylov vectors
n_krylov = 4
# Simulation time for each unitary
tau = 0.04
# Number of Trotter steps for time evolution
n_trotter_steps = 5
# Trotter order (1 for first-order, 2 for second-order)
trotter_order = 2
# Qubit Mapping
mapping = "jw"

backend = get_backend()

# Qubit operator for LiH
qu_op = f2q_mapping(mol_LiH.fermionic_hamiltonian, mapping, mol_LiH.n_active_sos,
                     mol_LiH.n_active_electrons, up_then_down=False, spin=mol_LiH.spin)

# Control qubit
c_q = count_qubits(qu_op)

# Operator that measures off-diagonal matrix elements i.e. 2|0><1|
zeroone = (QubitOperator(f"X{c_q}", 1) + QubitOperator(f"Y{c_q}", 1j))

# Controlled unitaries for each term in qu_op
c_qu = controlled_pauliwords(qubit_op=qu_op, control=c_q, n_qubits=5)

# Controlled time-evolution of qu_op with second-order Trotter-Suzuki decomposition
c_trott = trotterize(qu_op, time=tau, n_trotter_steps=n_trotter_steps, trotter_order=trotter_order, control=4)

# Generate multiple controlled-reference states.
reference_states = list()
reference_vecs = [[1, 1, 0, 0], [1, 0, 0, 1]]
for vec in reference_vecs:
    circ = vector_to_circuit(vec)
    gates = [Gate("C"+gate.name, target=gate.target, control=4) for gate in circ]
    reference_states += [Circuit(gates)]

# Calculate MRSQK
sab = np.zeros((n_krylov, n_krylov), dtype=complex)
hab = np.zeros((n_krylov, n_krylov), dtype=complex)
fhab = np.zeros((n_krylov, n_krylov), dtype=complex)

for a, b in product(range(n_krylov), range(n_krylov)):
    # Generate Ua and Ub unitaries with Trotter steps
    ua = reference_states[a%2] + c_trott * (a//2) if a > 1 else reference_states[a%2]
    ub = reference_states[b%2] + c_trott * (b//2) if b > 1 else reference_states[b%2]

    # Build circuit for off-diagonal overlap
    hab_circuit = Circuit([Gate("H", c_q)]) + ua + Circuit([Gate("X", c_q)]) + ub
    sab[a, b] = backend.get_expectation_value(zeroone, hab_circuit) / 2
    sab[b, a] = sab[a, b].conj()

    # Hamiltonian matrix element for f(H) = e^{-i H \tau}
    fhab[a, b] = backend.get_expectation_value(zeroone, hab_circuit + c_trott.inverse()) / 2
    print(f"Expectation value: {fhab[a,b]}")

    # Return statevector for faster calculation of Hamiltonian matrix elements
    _, initial_state = backend.simulate(hab_circuit, return_statevector=True)
    for i, (term, coeff) in enumerate(qu_op.terms.items()):

        # Append controlled-pauliword for each term in Hamiltonian and measure zeroone
        expect = coeff * backend.get_expectation_value(zeroone, c_qu[i], initial_statevector=initial_state) / 2

        # Add term to sum
        hab[a, b] += expect

# Solve the eigenvalue problem HV=SVE
e, v = eigh(hab, sab)
print(f"The HV=SVE energies are {e}")

# Solve for f(H)V=SVf(E) energies using f(H)
e = eigvals(fhab, sab)
print(f"The f(H)V=SVf(E) energies are {np.arctan2(np.imag(e), np.real(e))/tau}")

# Record algorithm resources used
algorithm_resources["mrsqk"] = dict()
algorithm_resources["mrsqk"]["qubit_hamiltonian_terms"] = len(qu_op.terms)
algorithm_resources["mrsqk"]["circuit_2qubit_gates"] = hab_circuit.counts.get("CNOT", 0)
algorithm_resources["mrsqk"]["n_post_terms"] = n_krylov**2



In [ ]:

def calculate_overlap(state1, state2):
    return np.abs(np.dot(np.conjugate(state1), state2))**2

overlap_ground = calculate_overlap(result, evolved_ground_state)
overlap_excited = calculate_overlap(excited_state, evolved_excited_state)

Plot the time occupancy. (we can make this prettier)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tangelo.toolboxes.operators import QubitOperator

# Assuming your previous Trotter-Suzuki implementation works and evolves states as you desire.
# This includes functions like trotterize, f2q_mapping, and the definition of your Hamiltonian (qu_op).

# Parameters for time evolution
time_steps = 100  # Number of time steps
total_time = 10.0  # Total time for evolution
tau = total_time / time_steps  # Time step for each slice

# Lists to store the ground and excited state occupancies over time
groundstate_occupancy = []
excitedstate_occupancy = []

# Time values for plotting
time_values = np.linspace(0, total_time, time_steps)

# Function to calculate the overlap between two quantum states
def calculate_overlap(state1, state2):
    return np.abs(np.dot(np.conjugate(state1), state2)) ** 2

# Manually evolve states using the previously defined Trotter-Suzuki time evolution
for t in time_values:
    # Assuming ground_state and excited_state are defined from VQE or initial states
    evolved_ground_state = trotterize(qu_op, time=t, n_trotter_steps=1, trotter_order=1).evolve(ground_state, tau)
    evolved_excited_state = trotterize(qu_op, time=t, n_trotter_steps=1, trotter_order=1).evolve(excited_state, tau)

    # Calculate overlap with original states
    overlap_ground = calculate_overlap(ground_state, evolved_ground_state)
    overlap_excited = calculate_overlap(excited_state, evolved_excited_state)

    # Store occupancies (overlaps)
    groundstate_occupancy.append(overlap_ground)
    excitedstate_occupancy.append(overlap_excited)

# Plotting the results
plt.plot(time_values, groundstate_occupancy, label="Ground State")
plt.plot(time_values, excitedstate_occupancy, label="Excited State")
plt.xlabel("Time")
plt.ylabel("Occupancy")
plt.legend()
plt.show()
